In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
ls -la

total 71301
-rw------- 1 root root    18546 Apr 18 09:03 'HW2: Profile & optimise PairwiseCounter.ipynb'
-rw------- 1 root root    16961 Apr 18 09:00 'HW2: Profile & optimise PairwiseCounter_optimize.ipynb'
-r-------- 1 root root 72975044 Apr  1 17:30  product_pairwise_counter.txt
-r-------- 1 root root      151 Mar 25 20:14 'Кирилл Ионкин - Предложения о работе с ODS.gdoc'


## Краткое описание поставленной задачи

Вам дан реальный продакшн-код, в котором имеются серьёзные узкие места по времени работы. **Ваша задача:** разобраться, как он устроен, **как можно точнее** найти узкие места при помощи профилировщика и устранить их, **сохраняя интерфейс** (т.е. нельзя менять методы класса `PairwiseCounter` или его сигнатуры). 

Нужные данные можно скачать из записи про это задание в Google Classroom (они довольно большие, поэтому рекомендую загрузить их один раз на свой Google Drive, а затем примонитровать его, чтобы ускорить процесс попадания данных на Colab-инстанс).

In [1]:
%%writefile pairwise_counter.py
from typing import Tuple, Iterable, Any, Dict, Optional, NamedTuple
 
import numpy as np
 
from scipy import sparse
 
 
# need for log(0) where pair_count = 0
# do not affect results
EPS = 1e-100
 
 
class Stats(NamedTuple):
    pair_count: float
    count_1: float
    count_2: float
    total: float
 
 
class PairwiseCounter:
 
    def __init__(
            self,
            counts_matrix: sparse.csr_matrix,
            index_mapper: Dict[Any, int],
            total_key: Any,
    ):
        """
        Class for calculating some pair statistics.
        :param counts_matrix: sparse matrix of pairs
        :param index_mapper: dict from key to index in matrix
        :param total_key: key to count size of the data by line
        (total_key, total_key, value)
        """
        self.counts_matrix = counts_matrix
        self.index_mapper = index_mapper
        self.total_key = total_key
        total_index = index_mapper[total_key]
        self.total = self.counts_matrix[total_index, total_index]
 
    def get_stats(self, key_1: Any, key_2: Any) -> Optional[Stats]:
        index_1 = self.index_mapper.get(key_1)
        index_2 = self.index_mapper.get(key_2)
 
        if index_1 is None or index_2 is None:
            return None
 
        pair_count = self.counts_matrix[index_1, index_2]
        count_1 = self.counts_matrix[index_1, index_1]
        count_2 = self.counts_matrix[index_2, index_2]
 
        if not count_1 or not count_2:
            return None
 
        return Stats(
            pair_count=float(pair_count),
            count_1=float(count_1),
            count_2=float(count_2),
            total=float(self.total),
        )
 
    def calculate_pmi(self, key_1: Any, key_2: Any) -> Optional[float]:
        """
        Calculates by formula: PMI
        PMI = log(p(x,y)/(p(x)p(y)))
        :param key_1: key 1
        :param key_2: key 2
        :return: weighted PMI
        """
 
        stats = self.get_stats(key_1, key_2)
        if stats is None:
            return None
        return (
            np.log(stats.pair_count + EPS)
            + np.log(stats.total)
            - np.log(stats.count_1)
            - np.log(stats.count_2)
        )
 
    def to_dict(self) -> Dict[str, Any]:
        counts_matrix_dict = dict(
            data=self.counts_matrix.data.tolist(),
            indices=self.counts_matrix.indices.tolist(),
            indptr=self.counts_matrix.indptr.tolist(),
            shape=self.counts_matrix.shape,
        )
        return dict(
            counts_matrix=counts_matrix_dict,
            index_mapper=self.index_mapper,
            total_key=self.total_key,
        )
 
    @staticmethod
    def from_dict(params_dict: Dict[str, Any]):
        counts_matrix = sparse.csr_matrix(
            (
                params_dict['counts_matrix']['data'],
                params_dict['counts_matrix']['indices'],
                params_dict['counts_matrix']['indptr'],
            ),
            shape=params_dict['counts_matrix']['shape'],
        )
        return PairwiseCounter(
            counts_matrix=counts_matrix,
            index_mapper=params_dict['index_mapper'],
            total_key=params_dict['total_key'],
        )

Writing pairwise_counter.py


## Импорт библиотек

In [10]:
import collections
import json
import typing as tp

from tqdm.auto import tqdm

from pairwise_counter import PairwiseCounter

## Загрузка данных

Предполагается, что данные лежат в папке `/content`. Если вы примонтировали папку со своего Google Drive, измените пути соответствующим образом.

In [17]:
%%timeit
with open('product_pairwise_counter.txt', 'r', encoding='utf8') as infile:
    pairwise_counter = PairwiseCounter.from_dict(json.load(infile))

1 loop, best of 5: 3.37 s per loop


In [16]:
%%timeit
product_ids = [
    product_id 
    for product_id in pairwise_counter.index_mapper.keys()
    if product_id != pairwise_counter.total_key
]

1000 loops, best of 5: 896 µs per loop


## Предподсчёт кандидатов для ранжирования

Класс `PairwiseCounter` нужен для того, чтобы предпосчитать кандидатов для ранжирования в рекомендательной системе. А именно: для каждого товара он находит `N` наиболее часто встречающихся вместе с ним в одной корзине (здесь `N = 10`). 

Тривиальная реализация этого функционала подразумевает два вложенных цикла. Это, очевидно, небыстро: сейчас  подсчёт занимает примерно 2 часа. **Ваша задача:** добиться максимального прироста производительности, изменяя только реализацию методов класса `PairwiseCounter` и функцию, которая вычисляет списки кандидатов (её можно менять как угодно, лишь бы ответы совпадали).

Методы `PairwiseCounter` нельзя менять по той причине, что он используется не только здесь, но и при вычислении признакового описания продуктовых корзин. Поэтому придётся работать с тем, что есть. 

В практике Яндекс.Лавки ускорение одного лишь класса `PairwiseCounter` значимо понизило долю таймаутов рекомендательной системы на треть, что дало дополнительные сотни тысяч рублей прибыли ежемесячно.

In [22]:
%%timeit

MAX_TOP_CANDIDATES: int = 10
most_co_occurring_products: tp.Dict[str, tp.List[str]] = dict()
 
for key_1 in tqdm(product_ids[:100], desc='outer loop'):
    candidates: tp.List[tp.Tuple[str, float]] = []
    for key_2 in product_ids[:100]: 
        if key_1 == key_2:
            continue
 
        pmi = pairwise_counter.calculate_pmi(key_1, key_2)
        if pmi is None:
            continue
 
        candidates.append((key_2, pmi))
 
    top_candidates = sorted(
        candidates, 
        key=lambda p: p[1], 
        reverse=True
    )[:MAX_TOP_CANDIDATES]
    most_co_occurring_products[key_1] = [
        product_id
        for product_id, pmi in top_candidates
    ]


1 loop, best of 5: 1.09 s per loop


In [23]:
with open('most_co_occurring_products.txt', 'w') as outfile:
    json.dump(most_co_occurring_products, outfile)

**ВАЖНО:** Прикрепите скриншоты со сравнением времени работы исходного решения и вашего. Суть ваших правок должна быть понятна преподавателям без необходимости запускать ваш код!